In [5]:
import pandas as pd
import happybase

In [6]:
connection = happybase.Connection('localhost')
# table_name = 'covid_table'
# table = connection.table(table_name)

In [7]:
df_confirmed = pd.read_csv('time_series_covid19_confirmed_global.csv')
df_deaths = pd.read_csv('time_series_covid19_deaths_global.csv')
df_recovered = pd.read_csv('time_series_covid19_recovered_global.csv')

# display(df_confirmed)
# display(df_deaths)
# display(df_recovered)

In [8]:
data_columns=df_confirmed.columns
country_columns=data_columns[:4]
date_columns=data_columns[4:]
country_columns

df_confirmed_unpivoted = df_confirmed.melt(id_vars=country_columns, var_name='date', value_name='confirmed')
df_deaths_unpivoted = df_deaths.melt(id_vars=country_columns, var_name='date', value_name='deaths')
df_recovred_unpivoted = df_recovered.melt(id_vars=country_columns, var_name='date', value_name='recovred')


# display(df_confirmed_unpivoted)
# display(df_deaths_unpivoted)
# display(df_recovred_unpivoted)


In [9]:

df_concatenated = pd.concat([df_confirmed_unpivoted, df_deaths_unpivoted["deaths"], df_recovred_unpivoted["recovred"]], axis=1)

df_concatenated.drop('Province/State', axis=1, inplace=True)
df_concatenated = df_concatenated.fillna(0)
df_concatenated['date'] = pd.to_datetime(df_concatenated['date'])

display(df_concatenated)

# df_concatenated.info()


C:\Users\vmazabraud\AppData\Local\Temp\ipykernel_21404\2874213863.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_concatenated['date'] = pd.to_datetime(df_concatenated['date'])


,Country/Region,Lat,Long,date,confirmed,deaths,recovred
0,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0.0
1,Albania,41.153300,20.168300,2020-01-22,0,0,0.0
2,Algeria,28.033900,1.659600,2020-01-22,0,0,0.0
3,Andorra,42.506300,1.521800,2020-01-22,0,0,0.0
4,Angola,-11.202700,17.873900,2020-01-22,0,0,0.0
...,...,...,...,...,...,...,...
330322,West Bank and Gaza,31.952200,35.233200,2023-03-09,703228,5708,0.0
330323,Winter Olympics 2022,39.904200,116.407400,2023-03-09,535,0,0.0
330324,Yemen,15.552727,48.516388,2023-03-09,11945,2159,0.0
330325,Zambia,-13.133897,27.849332,2023-03-09,343135,4057,0.0


In [10]:

# Définir une fonction pour insérer les données dans HBase
def insert_row(row):
    row_key = str(row.name)  # Utiliser l'index de la ligne comme clé de ligne
    data = {
        b'info:Country/Region': str(row['Country/Region']).encode(),
        b'info:date': str(row['date']).encode(),
        b'info:Lat': str(row['Lat']).encode(),
        b'info:Long': str(row['Long']).encode(),
        b'info:confirmed': str(row['confirmed']).encode(),
        b'info:deaths': str(row['deaths']).encode(),
        b'info:recovred': str(row['recovred']).encode()
    }
    table.put(row_key, data)
 
# Vérification de l'existence de la table
try:
    if b'covid_table' not in connection.tables():
        print("La covid n'existe pas, création...")
        connection.create_table(
            'covid_table',  # Nom de la table
            {column_family: dict()}  # Spécifiez la famille de colonnes
        )
 
    # Insertion des données dans la table
    with connection.table(b'covid_table').batch(batch_size=1000) as table:
        df_concatenated.apply(insert_row, axis=1)
        print("insertion des données terminée")
        pass
 
finally:
    connection.close()  # Fermeture de la connexion à la base de données

La covid n'existe pas, création...


NameError: name 'column_family' is not defined